# Bad Team's Final Submission for AMLC 2020 <a name='top'><a/>
## Last update: Nov 6, 2020

# Outline
1. [Module Import and Data Preprocessing](#m1)
2. [LSTM Seq2Seq Modeling](#m2)
3. [Predict for Phase 1](#m3)
4. [Predict for Phase 2](#m4)
5. [Evaluation by Committee](#m5)

# 0. Module Import and Data Preprocessing <a name='m1'></a> 
[back to Top](#top)

In [1]:
import pandas as pd
import numpy as np
import math

#from tqdm import tqdm

from pathlib import Path

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

import boto3
import io
import os

#from axp.ml.utils.fs.file_system import FileSystemFactory
#from mls_mdk.utils.file_protocol_utils import FileProtocolUtility

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data


#import seaborn as sns
#from matplotlib import pyplot as plt
#%matplotlib inline

In [2]:
import random

def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=1903)

In [3]:
# ref: https://github.com/Mrpatekful/swats/blob/master/swats/optim.py
class SWATS(torch.optim.Optimizer):
    r"""Implements Switching from Adam to SGD technique. Proposed in
    `Improving Generalization Performance by Switching from Adam to SGD`
    by Nitish Shirish Keskar, Richard Socher (2017).
    The method applies Adam in the first phase of the training, then
    switches to SGD when a criteria is met.
    Implementation of Adam and SGD update are from `torch.optim.Adam` and
    `torch.optim.SGD`.
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False, verbose=False, 
                 nesterov=False):
        if not 0.0 <= lr:
            raise ValueError(
                "Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError(
                "Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(
                "Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(
                "Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps, phase='ADAM',
                        weight_decay=weight_decay, amsgrad=amsgrad,
                        verbose=verbose, nesterov=nesterov)

        super().__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)
            group.setdefault('nesterov', False)
            group.setdefault('verbose', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): 
                A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for w in group['params']:
                if w.grad is None:
                    continue
                grad = w.grad.data

                if grad.is_sparse:
                    raise RuntimeError(
                        'Adam does not support sparse gradients, '
                        'please consider SparseAdam instead')

                amsgrad = group['amsgrad']

                state = self.state[w]

                # state initialization
                if len(state) == 0:
                    state['step'] = 0
                    # exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(w.data)
                    # exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(w.data)
                    # moving average for the non-orthogonal projection scaling
                    state['exp_avg2'] = w.new(1).fill_(0)
                    if amsgrad:
                        # maintains max of all exp. moving avg.
                        # of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(w.data)

                exp_avg, exp_avg2, exp_avg_sq = \
                    state['exp_avg'], state['exp_avg2'], state['exp_avg_sq'],

                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                if group['weight_decay'] != 0:
                    grad.add_(group['weight_decay'], w.data)

                # if its SGD phase, take an SGD update and continue
                if group['phase'] == 'SGD':
                    if 'momentum_buffer' not in state:
                        buf = state['momentum_buffer'] = torch.clone(
                            grad).detach()
                    else:
                        buf = state['momentum_buffer']
                        buf.mul_(beta1).add_(grad)
                        grad = buf

                    grad.mul_(1 - beta1)
                    if group['nesterov']:
                        grad.add_(beta1, buf)

                    w.data.add_(-group['lr'], grad)
                    continue

                # decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # maintains the maximum of all 2nd
                    # moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * \
                    (bias_correction2 ** 0.5) / bias_correction1

                p = -step_size * (exp_avg / denom)
                w.data.add_(p)

                p_view = p.view(-1)
                pg = p_view.dot(grad.view(-1))

                if pg != 0:
                    # the non-orthognal scaling estimate
                    scaling = p_view.dot(p_view) / -pg
                    exp_avg2.mul_(beta2).add_(1 - beta2, scaling)

                    # bias corrected exponential average
                    corrected_exp_avg = exp_avg2 / bias_correction2

                    # checking criteria of switching to SGD training
                    if state['step'] > 1 and \
                            corrected_exp_avg.allclose(scaling, rtol=1e-6) and \
                            corrected_exp_avg > 0:
                        group['phase'] = 'SGD'
                        group['lr'] = corrected_exp_avg.item()
                        if group['verbose']:
                            print('Switching to SGD after '
                                  '{} steps with lr {:.5f} '
                                  'and momentum {:.5f}.'.format(
                                      state['step'], group['lr'], beta1))

In [4]:
import pickle
def dump_pkl(model, scaler, sequence_in, filename):
    """prediction: df or numpy array; valid_loss: float; filename: str"""
    output = dict()
    output['model'] = model
    output['scaler'] = scaler
    output['sequence_in'] = sequence_in
    with open(filename,  'wb') as f:
        pickle.dump(output, f)

def load_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [5]:
data_path = %pwd
data_path = Path(data_path+"/data")

In [6]:
df_train = pd.read_csv(Path(data_path, 'df_train.csv'), index_col='INTERVAL')
df_train.sort_index(inplace=True)

In [7]:
df_phase2 = pd.read_csv(Path(data_path, 'df_phase2.csv'), index_col='INTERVAL')
df_phase2.sort_index(inplace=True)

In [8]:
n_features=len(df_train.columns)
n_features

20

# 0.1 Data Smoothing
[back to Top](#top)

In [9]:
def data_smooth(df, method='rolling_average', window=20):
    """Smooth the training data"""
    if method == 'rolling_average':
        return df.rolling(window).mean()

# 0.2 Data Preparation
[back to Top](#top)

In [10]:
# multivariate multi-step data preprocess (i.e. convert from time series problem to supervised learning problem)

# set the number of future hours to predict
hours_to_predict = 8

# choose the number of time steps to be read in and predicted
n_steps_in, n_steps_out = 96, int(4*hours_to_predict)

In [11]:
# 1. validation strategy: rolling window with sliding distance=1 DataPoint
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def rw_validation(df, train_size, valid_size, sliding_distance=1):
    # normalization 
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df)
    
    # convert into input/output
    X, y = split_sequences(np.array(df_scaled), train_size, valid_size)

    # break scaled data into train and validation datasets
    # validation dataset will be comprised of approximately the last 30 days of 2018
    n_days_val = 30
    n_steps_val = n_days_val*24*4
    X_train, y_train = X[:-n_steps_val], y[:-n_steps_val]
    X_val, y_val = X[-n_steps_val:], y[-n_steps_val:]

    # the dataset knows the number of features
    #n_features = X_train.shape[2]
    
    return X_train, y_train, X_val, y_val, scaler

In [12]:
def prep_smooth_data(df, n_steps_in=[96], n_steps_out=32, window_size=20):
    df_list = list()
    output_list = list()
    for step in n_steps_in:
        dfs = data_smooth(df, window=window_size).dropna()
        df_list.append(dfs)
        x_train, y_train, x_valid, y_valid, scaler = rw_validation(dfs, step, n_steps_out)
        output_list.append([x_train, y_train, x_valid, y_valid, scaler, step])
    return output_list

In [13]:
# get a good validation set (only include 12AM-8AM data on each day in last 30 days of 2018)
def make_p1_validation(df, n_days_val=30, n_steps_in=n_steps_in, n_steps_out=n_steps_out, all_as_valid=False):
    one_day = 24*4
    
    n_feature = len(df.columns)
    
    if all_as_valid:
        # on all training data
        start = one_day
    else:
        #only validation data
        start = len(df)-(n_days_val+1)*one_day
        
    end = len(df)
    
    total_len = int((end-start)/one_day)
    X_valid = np.empty((total_len, n_steps_in, n_feature))
    y_valid = np.empty((total_len, n_steps_out, n_feature))
    
    count = 0

    for i in range(start, end, one_day):
        #print("i=", i, "count= ", count, " ts: ", df.index[i])
        #print("X:", df.iloc[i-n_steps_in:i,:])
        #print("Y:", df.iloc[i:i+n_steps_out,:])
        X_valid[count] = df.iloc[i-n_steps_in:i,:].values
        y_valid[count] = df.iloc[i:i+n_steps_out,:].values
        count = count+1
    
    #print("total count: ", count)
    return X_valid, y_valid

In [14]:
#dataset_list = prep_smooth_data(df_train, n_steps_in=[96])

In [15]:
#X_valid, y_valid = make_p1_validation(df_train, all_as_valid=True)

# 1. LSTM-Seq2Seq Model <a name='m2'></a>
[back to Top](#top)

## 1.1 Encoder

In [16]:
class Encoder(nn.Module):
    def __init__(self, hidden_size, in_channel=20, sequence_in=64, sequence_out=32, dropout=0, lstm_layer_number=1):
        super().__init__()
        #n_directions = 1
        
        self.in_channel = in_channel
        self.sequence_in = sequence_in
        self.sequence_out = sequence_out
        self.hidden_size = hidden_size
        self.lstm_layer_number = lstm_layer_number
        self.dropout = nn.Dropout(dropout)

        self.lstm = torch.nn.LSTM(in_channel, hidden_size, num_layers=lstm_layer_number, batch_first=True, dropout=dropout)
    
    def forward(self, x):
        """Forward compute based on src sequence"""
        batch_size = x.shape[0]
        # x: [batch_size, sequence_in, in_channel]
        
        #apply lstm layer in encoder
        init_hidden_cell = (torch.zeros(self.lstm_layer_number,batch_size,self.hidden_size), 
                       torch.zeros(self.lstm_layer_number,batch_size,self.hidden_size))
        outputs, (hidden, cell) = self.lstm(x, init_hidden_cell)
        #ouputs: [ batch size,  sequence_in, hidden_size * n_directions]
        #hidden, cell: [batch size,  lstm_layer_number* n directions, hidden_size]
        
        return hidden, cell
        

## 1.2 Decoder

In [17]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, in_channel=20, sequence_in=64, sequence_out=32, dropout=0, lstm_layer_number=1):
        super().__init__()
        #n_directions = 1
        
        self.in_channel = in_channel
        self.sequence_in = sequence_in
        self.sequence_out = sequence_out
        self.hidden_size = hidden_size
        self.lstm_layer_number = lstm_layer_number
        self.dropout = nn.Dropout(dropout)
        
        self.lstm = torch.nn.LSTM(in_channel, hidden_size, num_layers=lstm_layer_number, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size, in_channel)
    
    def forward(self, x, hidden, cell):
        # x: [batch_size, in_channel]
        x = x.unsqueeze(1)
        
        # x: [batch_size, 1, in_channel]
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        # output: [batch_size, 1, hidden_size]
        # hidden, cell:
        
        prediction = self.fc_out(output.squeeze(1))
        # prediction: [batch_size, in_channel]
        
        return prediction, hidden, cell
        

## 1.3. Seq2Seq Model

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, hidden_size, device, in_channel=20, sequence_in=64, sequence_out=32, dropout=0, lstm_layer_number=1):
        super().__init__()
        
        self.in_channel = in_channel
        self.sequence_in = sequence_in
        self.sequence_out = sequence_out
        self.device = device
        
        self.encoder = Encoder(hidden_size, in_channel=in_channel, sequence_in=sequence_in, sequence_out=sequence_out, dropout=dropout, lstm_layer_number=lstm_layer_number)
        self.decoder = Decoder(hidden_size, in_channel=in_channel, sequence_in=sequence_in, sequence_out=sequence_out, dropout=dropout, lstm_layer_number=lstm_layer_number)
    
    def forward(self, x):
        # x is source sequence defined in training dataset
        # x: [batch_size, sequence_in,  in_channel]
        batch_size = x.shape[0]
        
        # Step-0: intialize the outputs
        # outputs: [batch_size, sequence_out, in_channel]
        outputs = torch.zeros(self.sequence_out, batch_size, self.in_channel).to(self.device)
        
        # Step-1: Encoder for input sequence
        hidden, cell = self.encoder(x)
        
        # Step-2: Step-wise apply decoder until sequence_out is reached
        # define initial input to decoder
        input = x[:,-1,:]
        #print(input.shape)
        
        for i in range(self.sequence_out):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[i] = output
            
            input = output
        
        return outputs.transpose(0,1)

## 1.4 Train, Valid and Infer Functions

In [19]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

## 1.5 Dataset Constructor

In [20]:
class Dataset:
    # targets=None, set up for test dataset
    # target is not None, set up for train dataset
    def __init__(self, features, targets=None):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        if self.targets is not None:
            return {
                'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
                'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
            }
        else: 
            return {
                'x' : torch.tensor(self.features[idx, :], dtype=torch.float)          
            }

## 1.6 LSTM_Seq2Seq Model Object

In [21]:
class lstm_seq_model:
    def __init__(self, hidden_size=128, seed=2020, device=('cuda' if torch.cuda.is_available() else 'cpu'), optimizer='swats',  
                 epochs=10,  batch_size=128, early_stopping_steps=10, early_stop=True, dropout=0, sequence_in=96, sequence_out=32,
                  activation='relu', learning_rate=1e-3, weight_decay=1e-5, lstm_layer_number=1, model_name='CNN.LSTM.BestModel.pth'):
        self.hidden_size  = hidden_size
        self.seed = seed
        self.device = device
        self.epochs = epochs
        self.batch_size = batch_size
        self.early_stopping_steps = early_stopping_steps
        self.early_stop = early_stop
        self.sequence_in = sequence_in
        self.sequence_out = sequence_out
        
        self.optimizer = optimizer
        
        self.dropout = dropout
        self.activation = activation
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.lstm_layer_number = lstm_layer_number
        self.model_name = model_name
        
        self.model = Seq2Seq(self.hidden_size, self.device, sequence_in=self.sequence_in, sequence_out=self.sequence_out, lstm_layer_number=self.lstm_layer_number, dropout=dropout).to(device)
        self.best_model = Seq2Seq(self.hidden_size, self.device, sequence_in=self.sequence_in, sequence_out=self.sequence_out, lstm_layer_number=self.lstm_layer_number, dropout=dropout).to(device)
      
    def set_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def fit(self, X_train, y_train, X_valid, y_valid):
        train_dl = torch.utils.data.DataLoader(Dataset(X_train, y_train), batch_size=self.batch_size, shuffle=True, num_workers=1)
        valid_dl = torch.utils.data.DataLoader(Dataset(X_valid, y_valid), batch_size=self.batch_size, shuffle=False, num_workers=1)
        
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        elif self.optimizer == 'swats':
            optimizer = SWATS(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                      max_lr=1e-2, epochs=self.epochs, steps_per_epoch=len(train_dl))
    
        
        optimize_loss = nn.L1Loss()  #mae
        #optimize_loss = nn.L2Loss() #mse
        evaluate_loss = nn.L1Loss()
        best_loss = np.inf

        #for epoch in tqdm(range(epochs)):
        for epoch in range(self.epochs):

            train_loss = train_fn(self.model, optimizer, scheduler, optimize_loss, train_dl, self.device)
            valid_loss, valid_preds = valid_fn(self.model, evaluate_loss, valid_dl, self.device)
            print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")

            if valid_loss < best_loss:
                best_loss = valid_loss
                #update best_model
                self.best_model.load_state_dict(self.model.state_dict())
                #reset early_step to make sure only do early stop for continuous valid_loss > best_loss
                early_step = 0 

            elif(self.early_stop == True):
                early_step += 1
                if (early_step >= self.early_stopping_steps):
                    break
                    
    def save_model(self):
        torch.save(self.best_model.state_dict(), self.model_name)
    
    def predict(self, X_test):
        test_dl = torch.utils.data.DataLoader(Dataset(X_test), batch_size=1, shuffle=False, num_workers=1)
        return inference_fn(self.best_model, test_dl, self.device) 

## 1.7 Training LSTM Seq2Seq Model

In [22]:
def full_train(dataset_list, hidden_size=128, epochs=60, seeds=[1903, 1], df_train=df_train, optimizer='swats', mode='single'):
    
    model_list = list()
    scaler_list = list()
    seq_in_list = list()
    X_valid_list = list()
    y_valid_list = list()
    
    for seed in seeds:
        seed_everything(seed=seed)
        
        for dataset in dataset_list:
            if mode == 'single':
                model = lstm_seq_model(hidden_size=hidden_size, epochs=epochs, optimizer=optimizer, sequence_in=dataset[5], sequence_out=32, model_name='smooth.test.lstm.pth' )
                model.fit(dataset[0], dataset[1], dataset[2], dataset[3])
            else:
                model = lstm_seq_model(hidden_size=hidden_size, epochs=epochs, optimizer='adam', sequence_in=dataset[5], sequence_out=32, model_name='smooth.test.lstm.pth' )
                model.fit(dataset[0], dataset[1], dataset[2], dataset[3])
                
                model.set_optimizer('swats')
                model.fit(dataset[0], dataset[1], dataset[2], dataset[3])
            
            model_list.append(model)
            scaler_list.append(dataset[4])
            seq_in_list.append(dataset[5])
            
            X_valid, y_valid = make_p1_validation(df_train, n_steps_in=dataset[5], all_as_valid=True)
            X_valid_list.append(X_valid)
            y_valid_list.append(y_valid)
            
        
    return model_list, scaler_list, seq_in_list, X_valid_list, y_valid_list

In [23]:
#model_list, scaler_list, seq_in_list, X_valid_list, y_valid_list = full_train(dataset_list, seeds=[1], epochs=60, optimizer='adam')

In [24]:
#sw_model_list, sw_scaler_list, sw_seq_in_list, sw_X_valid_list, sw_y_valid_list = full_train(dataset_list, seeds=[1903], epochs=60, optimizer='swats')

## 1.8 Local Evaluation of Model Performance

In [25]:
def evaluate_valid(model_list, scaler_list, seq_in_list, X_valid_list, y_valid_list, n_steps_out=32, 
                   n_features=n_features, raw_valid=True):
    """Given a list of models, this function evaluate on true valid dataset; only after-scaled data is reported!"""
    eva_loss = nn.L1Loss()
    loss_list = list()
    pred_list = list()
    error_list = list()
    
    # transform the ground truth to raw data's scale
    #y_valid = scaler.inverse_transform(y_valid.reshape(-1,n_features))
    
    for j, model in enumerate(model_list):
        i_scaler = scaler_list[j]
        
        X_valid = X_valid_list[j]
        y_valid = y_valid_list[j]
        n_steps_in = seq_in_list[j]
        
        prediction = np.empty(y_valid.shape)
        val_errors = list()
        
        for i in range(len(X_valid)):
            # reshape validation samples and make individual prediction
            if raw_valid:
                X_val_step = i_scaler.transform(X_valid[i]).reshape((1, n_steps_in, n_features))
            else:
                X_val_step = X_valid[i].reshape((1, n_steps_in, n_features))
                
            y_preds = model.predict(X_val_step).reshape(n_steps_out,n_features)
            
            if raw_valid:
                #print("at i=",  i, " ",  y_valid[i])
                #print(i_scaler.inverse_transform(y_preds))
                error = mean_absolute_error(i_scaler.inverse_transform(y_preds), y_valid[i])
                #print(error)
                
            val_errors.append(error)
            prediction[i] = y_preds
        
        error_list.append(val_errors)
        loss_list.append(np.mean(val_errors))
        pred_list.append(prediction)
    return loss_list, pred_list, error_list

In [26]:
#loss, pred, err = evaluate_valid(model_list, scaler_list, seq_in_list, X_valid_list, y_valid_list)

In [27]:
#loss1, pred1, err1 = evaluate_valid(sw_model_list, sw_scaler_list, sw_seq_in_list, sw_X_valid_list, sw_y_valid_list)

In [28]:
def weight_average_valid(model_list, scaler_list, seq_in_list, X_valid_list, y_valid_list, n_steps_out=32, 
                   n_features=n_features,  weights=None):
    if weights is None:
        N = len(model_list)
        weights = [1/N for i in range(N)]
    print(f"weights: {weights}")
    
    
    pred_list = list()
    loss_list = list()
    error_list = list()
    
    for i, model in enumerate(model_list):
        i_scaler = scaler_list[i]
        X_valid = X_valid_list[i]
        y_valid = y_valid_list[i]
        n_steps_in = seq_in_list[i]
        
        i_prediction = np.empty(y_valid.shape)
        val_errors = list()
        
        for j in range(len(X_valid)):
            X_val_step = i_scaler.transform(X_valid[j]).reshape((1, n_steps_in, n_features))
            y_preds = model.predict(X_val_step).reshape(n_steps_out,n_features)
            y_preds = i_scaler.inverse_transform(y_preds)
            error = mean_absolute_error(y_preds, y_valid[j])
            
            val_errors.append(error)
            i_prediction[j] = y_preds
        
        error_list.append(val_errors)
        pred_list.append(i_prediction)
        loss_list.append(np.mean(val_errors))
    

    # Weighted average of prediction
    prediction = np.average(pred_list, axis=0, weights=weights)

    # compute averaged MAE after weighted averagee
    fin_errors=list()
    for j in range(len(y_valid_list[0])):
        fin_error = mean_absolute_error(prediction[j], y_valid_list[0][j])
        fin_errors.append(fin_error)

    return prediction, np.mean(fin_errors), fin_errors

# 2. Predict for Phase 1 <a name='m3'></a>
[back to Top](#top)

## 2.1 Unpickle pre-trained models

In [29]:
#s1_model = load_pkl('models/seed1_adam_96in.pkl')
#s1903_model = load_pkl('models/seed1903_swats_96in.pkl')

In [30]:
#model_list  = [ s1_model['model'], s1903_model['model']]
#scaler_list = [ s1_model['scaler'], s1903_model['scaler']]
#seq_in_list = [ s1_model['sequence_in'], s1903_model['sequence_in']]

In [31]:
# Weighted Average for Local Validation
#fin_pred, fin_loss, fin_err = weight_average_valid(model_list, scaler_list, seq_in_list, [X_valid,X_valid], [y_valid,y_valid], weights=[0.7,0.3])
#fin_loss

## 2.2 Functions for Phase 1 Prediction

In [32]:
def predict_phase1(model, df_train, scaler, n_steps_in, n_steps_out=n_steps_out, n_features=n_features):
    df_test = df_train.iloc[-n_steps_in:, :]
    X_test = scaler.transform(df_test).reshape((1, n_steps_in, n_features))
    #X_test = np.swapaxes(X_test,1,2)
    y_preds = model.predict(X_test).reshape(n_steps_out, n_features)
    y_preds = scaler.inverse_transform(y_preds)
    df_preds = pd.DataFrame(y_preds, columns=df_train.columns)
    
    return df_preds

In [33]:
def weight_average_phase1(df_train, model_list, scaler_list, seq_in_list , n_steps_out=32, 
                   n_features=n_features, weights=None, output_format='pandas'):
    if weights is None:
        N = len(model_list)
        weights = [1/N for i in range(N)]
    print(f"weights: {weights}")
    
    pred_list = list()
    for i, model in enumerate(model_list):
        df_preds = predict_phase1(model, df_train, scaler_list[i], seq_in_list[i], n_steps_out=n_steps_out, 
                   n_features=n_features)
        pred_list.append(df_preds.values)
        
    prediction = np.average(pred_list, axis=0, weights=weights)
    
    if output_format == 'numpy':
        return prediction
    else:
        return pd.DataFrame(prediction)

# 3. Predict for Phase 2 <a name='m4'></a>
[back to Top](#top)

In [34]:
#  function to predict by a single model
#  reuse the script from starter notebook
def predict_phase2(df_phase2, model, scaler, n_steps_in, n_steps_out=32, n_features=20, n_steps_2019 = (31+28+31)*4*24):
    # scale dataset
    df_phase2_scaled = scaler.transform(df_phase2)
    # convert into input/output
    # in this case we don't need the outputs
    X_phase2, y_phase2 = split_sequences(np.array(df_phase2_scaled),
                                         n_steps_in, n_steps_out)
    # subset outputs to include only 2019 data
    X_2019 = X_phase2[-(n_steps_2019-n_steps_out+1):]
    # pre-allocate empty array for predictions
    Y_preds = np.empty([X_2019.shape[0], n_steps_out, n_features])

    # loop through Q1 2019 data input arrays, make predictions and write back
    # to multi-dimensional numpy array
    for step, data in enumerate(X_2019):
        X_2019_step = data.reshape((1, n_steps_in, n_features))
        y_preds = model.predict(X_2019_step).reshape(n_steps_out, n_features)
        Y_preds[step] = np.array(scaler.inverse_transform(y_preds))

    # output multi-dimensional numpy array of predictions
    return Y_preds

In [35]:
# function to render predictions for Q1 2019 data

# here we are specifying a single input: the Phase II dataframe
# teams are welcome to specify multiple function inputs for their custom function so long as they are properly defined and commented

def predict_Q1_2019(df_phase2, weights=[0.7,0.3], n_steps_out=32, n_features=20):
    
    # read pre-trained model from pickle files
    s1_model = load_pkl('models/seed1_adam_96in.pkl')
    s1903_model = load_pkl('models/seed1903_swats_96in.pkl')
    
    model_list  = [ s1_model['model'], s1903_model['model']]
    scaler_list = [ s1_model['scaler'], s1903_model['scaler']]
    seq_in_list = [ s1_model['sequence_in'], s1903_model['sequence_in']]
    
    # loop over all models and predict
    pred_list = list()
    #for i, model in enumerate(tqdm(model_list)):
    for i, model in enumerate(model_list):
        scaler = scaler_list[i]
        n_steps_in = seq_in_list[i]
        i_preds = predict_phase2(df_phase2, model, scaler, n_steps_in, n_steps_out=n_steps_out, n_features=n_features)
        pred_list.append(i_preds)
    
    # weight average over input models
    prediction = np.average(pred_list, axis=0, weights=weights)
    
    return prediction

In [36]:
Y_preds = predict_Q1_2019(df_phase2)

/Users/fyuan2/Virtualenv_Python/amlc2020/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [37]:
Y_preds.shape

(8609, 32, 20)

In [38]:
# check if first prediction in Y_preds is consisent with last submission in Phase1
def sanity_check(Y_preds, ref_pkl='last.submission.pkl'):
    with open(ref_pkl, 'rb') as f:
        last_sub = pickle.load(f)
    
    if np.array_equal(Y_preds[0], last_sub):
        return "Pass Sanity Check"
    else:
        return "Fail Sanity Check"

In [41]:
#sanity_check(Y_preds)

'Pass Sanity Check'

# 4. For Phase II evaluation by AMLC Committee -- NOT USED BY COMPETITORS <a name='m5'></a>
[back to Top](#top)

In [40]:
def Q1_2019_errors(Y_preds, Y_true):
    errors = np.empty(Y_preds.shape[0])
    for step, data in enumerate(Y_preds):
        errors[step] = mean_absolute_error(data, Y_true[step])
    return np.mean(errors)